# Public Sentiments & Chicago's Neighborhoods :: 
<h3>Final project for PPHA 30530 Computation for Public Policy</h3>

Exploring tweet sentiments regarding Chicago neighborhoods, and any correlations with Chicago's socioeconomic and crime data. The results will be shown in plots and on maps.

In [1]:
import tweepy
import time
import pandas as pd

In [2]:
import vaderSentiment  # THIS imports successfully, 
# but what matters is that I need the sentiment function defined in vaderSentiment to work towards the bottom of 
# this notebook. Using simply this import line doesn't do the trick.

In [4]:
from vaderSentiment import sentiment as vaderSentiment  
# This is the original suggested import line according to the package 
# But THIS gives me a "ImportError: cannot import name 'sentiment'"

ImportError: cannot import name 'sentiment'

In [8]:
from vaderSentiment.vaderSentiment import sentiment as vaderSentiment 
# offered as an alternative depending on install approach
# but THIS one also gives me an error ("SyntaxError: invalid syntax")

SyntaxError: invalid syntax (vaderSentiment.py, line 23)

# Step 1: Get the data

Downloading, cleaning up, and storing Twitter data in a csv file. 

In [2]:
# These four codes constitute your authorization to extract data from Twitter
consumer_token = 'qZMmvxcPLfwCixedks1m3jXGg'
consumer_secret = 'UFFyLdOlePodkPYvR6NR64N0SVinVsPezNb1IKg1hNXl06jy67'
access_token = '2382930698-0eCycGIeqv4SUmOvSINQbkhnb2v9hTPDlSpcb8q'
access_token_secret = 'eTmMeL9pj7pcxoupFDqDuNmzWSXUn8UQWrYeYAcKu8xyR'

# OAuth process, using the keys and tokens
auth = tweepy.OAuthHandler(consumer_token, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

# Creation of the actual interface, using authentication
# api = tweepy.API(auth)
api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

In [3]:
api.rate_limit_status('search')  # checks the status of my search request limit

{'rate_limit_context': {'access_token': '2382930698-0eCycGIeqv4SUmOvSINQbkhnb2v9hTPDlSpcb8q'},
 'resources': {'search': {'/search/tweets': {'limit': 180,
    'remaining': 180,
    'reset': 1457793990}}}}

<h3> Creating datasets of differing scopes: </h3>

<h5>Dataset 1:</h5>
    
    list of tweets within last 6-9 days
        NOTE: limit set by Twitter, not me, see https://dev.twitter.com/rest/public/search
    
    tweet mentions "englewood" AND "chicago"
        NOTE: There are two Community Areas inside Chicago with "Englewood" in the name. Thank goodness they're 
        adjacent to each other, so I assume that the unintended ambiguity won't significantly shift the analytical 
        results.  
    
    tweet originates within 15 miles of Chicago, IL
    
    tweet is in English

In [32]:
community_area_name1 = []
date_time1 = []
coordinates1 = []
num_retweet1 = []
num_liked1 = []
tweet_content1 = []
    
for tweet in tweepy.Cursor(api.search, 
                           q = "englewood",
                           geocode = "41.836944,-87.684722,15mi", 
                           lang = "en").items():
    community_area_name1.append('englewood')
    date_time1.append(tweet.created_at)
    coordinates1.append(tweet.coordinates)
    num_retweet1.append(tweet.retweet_count)
    num_liked1.append(tweet.favorite_count)
    tweet_content1.append(tweet.text)

dataset1 = pd.DataFrame({
        "Community Area Mentioned": community_area_name1,
        "Day and Time": date_time1,
        "Coordinates": coordinates1,
        "Number of Times Retweeted": num_retweet1,
        "Number of Times Liked": num_liked1,
        "Tweet Content": tweet_content1})

In [33]:
dataset1.shape  # includes 1 count for row with headers

(19, 6)

In [34]:
dataset1['Coordinates'][0]

{'coordinates': [-87.66115998, 41.78729154], 'type': 'Point'}

In [35]:
print(type(dataset1['Coordinates'][0]))

<class 'dict'>


In [24]:
# convert coordinates column, currently a dictionary, into string only
dataset1['Coordinates'] = dataset1['Coordinates'].apply(str)

In [25]:
print(type(dataset1['Coordinates'][0]))

<class 'str'>


In [29]:
dataset1.to_csv('dataset1.csv')

<h5>Dataset 2:</h5>
    
    list of tweets within last 6-9 days
    tweet mentions "englewood" AND "chicago"
    tweet originates from ANYWHERE
    tweet is in English

In [36]:
community_area_name2 = []
date_time2 = []
coordinates2 = []
num_retweet2 = []
num_liked2 = []
tweet_content2 = []

for tweet in tweepy.Cursor(api.search,
                           q = ["englewood", "chicago"],
                           lang = "en").items():
    community_area_name2.append('englewood')
    date_time2.append(tweet.created_at)
    coordinates2.append(tweet.coordinates)
    num_retweet2.append(tweet.retweet_count)
    num_liked2.append(tweet.favorite_count)
    tweet_content2.append(tweet.text)
    
dataset2 = pd.DataFrame({
        "Community Area Mentioned": community_area_name2,
        "Day and Time": date_time2,
        "Coordinates": coordinates2,
        "Number of Times Retweeted": num_retweet2,
        "Number of Times Liked": num_liked2,
        "Tweet Content": tweet_content2})

In [37]:
dataset2.shape # includes 1 count for row with headers

(416, 6)

In [38]:
# convert coordinates column, currently a dictionary, into string only
dataset2['Coordinates'] = dataset2['Coordinates'].apply(str)

In [39]:
dataset2.to_csv('dataset2.csv')

<h5>Dataset 3:</h5>
    
    list of tweets within last 6-9 days
    tweet mentions the name of one of 77 community areas
    tweet originates within 15 miles of Chicago, IL
    tweet is in English

In [150]:
# for list of community area names, import csv file
df_SES = pd.read_csv('https://raw.githubusercontent.com/yarikan/final_project_in_progress/master/Census_Data_-_Selected_socioeconomic_indicators_in_Chicago__2008___2012.csv?token=AQAvwKru9f-jT5DdlHHIx3sA0zKfj9cFks5W6IcxwA%3D%3D')
SES = df_SES.dropna()
# SES
# SES.columns
# SES['COMMUNITY AREA NAME']

In [152]:
community_area_name3 = []
date_time3 = []
coordinates3 = []
num_retweet3 = []
num_liked3 = []
tweet_content3 = []

for name in SES['COMMUNITY AREA NAME']:
    time.sleep(11)
    name.lower()
    for tweet in tweepy.Cursor(api.search,
                               q = name.lower(),
                               geocode = "41.836944,-87.684722,15mi",
                               lang = "en").items():
        community_area_name3.append(name.lower())
        date_time3.append(tweet.created_at)
        coordinates3.append(tweet.coordinates)
        num_retweet3.append(tweet.retweet_count)
        num_liked3.append(tweet.favorite_count)
        tweet_content3.append(tweet.text)

dataset3 = pd.DataFrame({
        "Community Area Mentioned": community_area_name3,
        "Day and Time": date_time3,
        "Coordinates": coordinates3,
        "Number of Times Retweeted": num_retweet3,
        "Number of Times Liked": num_liked3,
        "Tweet Content": tweet_content3})

In [153]:
dataset3.shape  # includes 1 count for row with headers

(1589, 6)

In [154]:
dataset3.head(2)

,Community Area Mentioned,Coordinates,Day and Time,Number of Times Liked,Number of Times Retweeted,Tweet Content
0,rogers park,"{'coordinates': [-87.6662979, 42.0074272], 'ty...",2016-03-12 01:02:57,0,0,"Hey, Donnie. Heard you bailed on your own part..."
1,rogers park,"{'coordinates': [-87.67733976, 42.01507579], '...",2016-03-12 00:41:15,1,0,Day 11 First Lady Friday: Go Michelle! #DePaul...


In [155]:
# convert coordinates column, currently a dictionary, into string only
dataset3['Coordinates'] = dataset3['Coordinates'].apply(str)

In [156]:
dataset3.to_csv('dataset3.csv')

<h3> Clean up datasets before proceeding to do sentiment analysis: </h3>

In [157]:
df1 = pd.read_csv('https://raw.githubusercontent.com/yarikan/final_project_in_progress/master/dataset1.csv?token=AQAvwMuX-qLmupa78p6_y3nhkWwqrjEOks5W7FZtwA%3D%3D')
df2 = pd.read_csv('https://raw.githubusercontent.com/yarikan/final_project_in_progress/master/dataset2.csv?token=AQAvwNC_wzbUUy0gbnpHm48odETa7Cdiks5W7FaCwA%3D%3D')
df3 = pd.read_csv('https://raw.githubusercontent.com/yarikan/final_project_in_progress/master/dataset3.csv?token=AQAvwJm8_ighu4wZm6POFm3kil4QlNG5ks5W7FarwA%3D%3D')

<h4> Dataset 1: </h4>

In [111]:
df1.shape

(19, 7)

In [112]:
df1.head(2)

,Unnamed: 0,Community Area Mentioned,Coordinates,Day and Time,Number of Times Liked,Number of Times Retweeted,Tweet Content
0,0,englewood,"{'coordinates': [-87.66115998, 41.78729154], '...",2016-03-12 06:44:37,0,0,"😎 @ Chicago (Englewood neighborhood), Illinois..."
1,1,englewood,"{'coordinates': [-87.64513733, 41.79094389], '...",2016-03-11 21:44:27,0,0,🃏🃏🃏 #watchthekid @ Streets Of Englewood https:...


In [113]:
df1['Coordinates'][0]

"{'coordinates': [-87.66115998, 41.78729154], 'type': 'Point'}"

In [114]:
# clean up coordinates column by extracting latitude and longitude:

cut_left = df1['Coordinates'].str.split('[').apply(pd.Series)
cut_left = cut_left[1]
# cut_left[0]

cut_right = cut_left.str.split(']').apply(pd.Series)
cut_right = cut_right[0]
# cut_right[0]

split_in2 = cut_right.str.split(',').apply(pd.Series)
#split_in2.columns

latitude = split_in2[0].apply(float)
longitude = split_in2[1].apply(float)

df1['Tweet Latitude'] = latitude
df1['Tweet Longitude'] = longitude

In [115]:
# convert Day and Time column (currently string) into datetime
df1['Day and Time']= pd.to_datetime(df1['Day and Time'])

# print(type(df1['Day and Time'][0]))
df1['Day and Time'][0]

Timestamp('2016-03-12 06:44:37')

In [116]:
del df1['Coordinates']
del df1['Unnamed: 0']

In [118]:
df1.shape

(19, 7)

In [117]:
df1.head(2)

,Community Area Mentioned,Day and Time,Number of Times Liked,Number of Times Retweeted,Tweet Content,Tweet Latitude,Tweet Longitude
0,englewood,2016-03-12 06:44:37,0,0,"😎 @ Chicago (Englewood neighborhood), Illinois...",-87.661160,41.787292
1,englewood,2016-03-11 21:44:27,0,0,🃏🃏🃏 #watchthekid @ Streets Of Englewood https:...,-87.645137,41.790944


In [119]:
df1.to_csv('df1_cleaned.csv')

<h4> Dataset 2: </h4>

In [120]:
df2.shape

(417, 7)

In [121]:
df2.head(2)

,Unnamed: 0,Community Area Mentioned,Coordinates,Day and Time,Number of Times Liked,Number of Times Retweeted,Tweet Content
0,0,englewood,None,2016-03-12 14:21:01,0,0,"Chicago (IL) WLS: 1 dead, 1 critical after pol..."
1,1,englewood,None,2016-03-12 14:16:23,0,0,@realDonaldTrump too bad #Chicago treated you ...


In [122]:
# clean up coordinates column by extracting latitude and longitude:

cut_left = df2['Coordinates'].str.split('[').apply(pd.Series)
cut_left = cut_left[1]
# cut_left[0]

cut_right = cut_left.str.split(']').apply(pd.Series)
cut_right = cut_right[0]
# cut_right[0]

split_in2 = cut_right.str.split(',').apply(pd.Series)
#split_in2.columns

latitude = split_in2[0].apply(float)
longitude = split_in2[1].apply(float)

df2['Tweet Latitude'] = latitude
df2['Tweet Longitude'] = longitude


# for tweets with no latitude and longitude, replace NaN with zeros

df2['Tweet Latitude'].fillna(0, inplace=True)
df2['Tweet Longitude'].fillna(0, inplace=True)

In [123]:
# convert Day and Time column (currently string) into datetime
df2['Day and Time']= pd.to_datetime(df2['Day and Time'])

# print(type(df2['Day and Time'][0]))
# df2['Day and Time'][0]

In [124]:
del df2['Coordinates']
del df2['Unnamed: 0']

In [125]:
df2.shape

(417, 7)

In [126]:
df1.head(2)

,Community Area Mentioned,Day and Time,Number of Times Liked,Number of Times Retweeted,Tweet Content,Tweet Latitude,Tweet Longitude
0,englewood,2016-03-12 06:44:37,0,0,"😎 @ Chicago (Englewood neighborhood), Illinois...",-87.661160,41.787292
1,englewood,2016-03-11 21:44:27,0,0,🃏🃏🃏 #watchthekid @ Streets Of Englewood https:...,-87.645137,41.790944


In [127]:
df2.to_csv('df2_cleaned.csv')

<h4> Dataset 3: </h4>

In [158]:
df3.shape

(1589, 7)

In [161]:
df3.head(2)

,Unnamed: 0,Community Area Mentioned,Coordinates,Day and Time,Number of Times Liked,Number of Times Retweeted,Tweet Content
0,0,rogers park,"{'coordinates': [-87.6662979, 42.0074272], 'ty...",2016-03-12 01:02:57,0,0,"Hey, Donnie. Heard you bailed on your own part..."
1,1,rogers park,"{'coordinates': [-87.67733976, 42.01507579], '...",2016-03-12 00:41:15,1,0,Day 11 First Lady Friday: Go Michelle! #DePaul...


In [162]:
# clean up coordinates column by extracting latitude and longitude:

cut_left = df3['Coordinates'].str.split('[').apply(pd.Series)
cut_left = cut_left[1]
# cut_left[0]

cut_right = cut_left.str.split(']').apply(pd.Series)
cut_right = cut_right[0]
# cut_right[0]

split_in2 = cut_right.str.split(',').apply(pd.Series)
#split_in2.columns

latitude = split_in2[0].apply(float)
longitude = split_in2[1].apply(float)

df3['Tweet Latitude'] = latitude
df3['Tweet Longitude'] = longitude

In [164]:
# convert Day and Time column (currently string) into datetime
df3['Day and Time']= pd.to_datetime(df3['Day and Time'])
# print(type(df3['Day and Time'][0]))
# df3['Day and Time'][0]

In [166]:
del df3['Coordinates']
del df3['Unnamed: 0']

In [167]:
df3.shape

(1589, 7)

In [168]:
df3.head(2)

,Community Area Mentioned,Day and Time,Number of Times Liked,Number of Times Retweeted,Tweet Content,Tweet Latitude,Tweet Longitude
0,rogers park,2016-03-12 01:02:57,0,0,"Hey, Donnie. Heard you bailed on your own part...",-87.666298,42.007427
1,rogers park,2016-03-12 00:41:15,1,0,Day 11 First Lady Friday: Go Michelle! #DePaul...,-87.677340,42.015076


In [169]:
df3.to_csv('df3_cleaned.csv')

# Step 2: Do sentiment analysis of tweets

I'm using VADER Sentiment Analysis (https://pypi.python.org/pypi/vaderSentiment/0.5) to apply a rule-based and lexical/dictionary approach to the sentiment analysis of the tweets I downloaded. VADER is sensitive to both the polarity  (positive/negative) and the intensity (on a scale from –4 to +4) of sentiments expressed in social media contexts. For example, the word "okay" has a positive valence of 0.9, "good" is 1.9, and "great" is 3.1, whereas "horrible" is –2.5, the frowning emoticon :( is –2.2, and "sucks" and it's slang derivative "sux" are both –1.5. 

In [3]:
df1_cleaned = pd.read_csv('https://raw.githubusercontent.com/yarikan/final_project_in_progress/master/df1_cleaned.csv?token=AQAvwJLRV9FI5rVlSRBrds9V9dSTCI8lks5W7xhgwA%3D%3D')
# df2_cleaned = pd.read_csv('https://raw.githubusercontent.com/yarikan/final_project_in_progress/master/df2_cleaned.csv?token=AQAvwJgTqkiyvo-qji8BW3OFzaI7rX1cks5W7xiUwA%3D%3D')
# df3_cleaned = pd.read_csv('https://raw.githubusercontent.com/yarikan/final_project_in_progress/master/df3_cleaned.csv?token=AQAvwL5u8yzFj8WjqkWj-3D5BPZ9_TUFks5W7ximwA%3D%3D')

In [4]:
tweets = df1_cleaned['Tweet Content']
# tweets

In [9]:
from vaderSentiment.vaderSentiment import sentiment

SyntaxError: invalid syntax (vaderSentiment.py, line 23)

In [6]:
for tweet in tweets:
    print(tweet)
    vs = vaderSentiment.vaderSentiment(tweet)
    print("\n\t" + str(vs))

😎 @ Chicago (Englewood neighborhood), Illinois https://t.co/LJ5J0yp5zd


AttributeError: module 'vaderSentiment' has no attribute 'vaderSentiment'

# Step 3: Show and tell
for mapping, consider also using plotli, (chloprograph?)

# Turning it in

You will demonstrate your project on March 18 between 6–8pm, and also include all code within the final_project folder (two or more scripts) in your GitHub repository. Please also include a file called README.txt or README.md that describes how to run your project, including any setup scripts that download data, etc. Finally, if you make any large data files in the course of running your website, please use a .gitignore file to exclude these from the repository.